<a href="https://colab.research.google.com/github/arnav13081994/ReinforcementLearning/blob/main/Cart_pole_my_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]
!pip install gymnasium[classic_control]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [108]:
import gymnasium as gym

def create_gym_env(env_name, **kwargs):
  env = gym.make(env_name, **kwargs)
  return env

env = create_gym_env('CartPole-v1')

In [109]:
# Create the Neural Net
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

class NeuralNet(nn.Module):

  def __init__(self, state_size, action_size, seed = 42):

    super().__init__()
    self.seed = torch.manual_seed(seed=seed)
    self.fc1 = nn.Linear(state_size, 256)
    self.fc2 = nn.Linear(256, 84)
    self.fc3 = nn.Linear(84, action_size)



  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x



In [110]:
# Create the Experience Replay Buffer
from collections import deque
import random

class ExperienceReplayBuffer:

  def __init__(self, capacity):
    self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    self.capcity = capacity
    self.memory = deque(maxlen=capacity)

  def push(self, experience):
    self.memory.append(experience)

  def sample(self, batch_size):
    learning_experiences = random.sample(self.memory, k = batch_size)

    states = torch.from_numpy(np.vstack([e[0] for e in learning_experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in learning_experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in learning_experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in learning_experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in learning_experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    return states, next_states, actions, rewards, dones





In [111]:
# Create the Agent Class
from torch.optim import Adam

learning_rate = 5e-4
buffer_size=int(1e5)
minibatch_size=100
discount_factor = 0.99
interpolation_parameter = 1e-3




class Agent:

  def __init__(self, state_size, action_size):
    self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    self.action_size = action_size
    self.state_size = state_size
    self.local_qnetwork = NeuralNet(state_size, action_size)
    self.target_qnetwork = NeuralNet(state_size, action_size)
    self.t_step = 0
    self.optimiser = Adam(self.local_qnetwork.parameters(), lr=learning_rate)
    self.memory = ExperienceReplayBuffer(capacity=buffer_size)



  def step(self, state, action, reward, next_state, done):
    self.memory.memory.append((state, action, reward, next_state, done))
    self.t_step = (self.t_step + 1) % 4
    if self.t_step == 0:
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(minibatch_size)
        self.learn(experiences, discount_factor)


  def act(self, state, epsilon):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    predicted_qvalues = self.local_qnetwork(state)
    if random.random() > epsilon:
      return np.argmax(predicted_qvalues.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))


  def learn(self, experiences, discount_factor):
    states, next_states, actions, rewards, dones = experiences
    q_target_next = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_target = rewards + discount_factor * (1-dones)*q_target_next
    q_predicted = self.local_qnetwork(states).gather(1, actions)
    # loss = F.mse_loss(q_predicted, q_target)
    loss = F.smooth_l1_loss(q_predicted, q_target)

    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)



  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_params, local_params in zip(target_model.parameters(), local_model.parameters()):
      target_params.data.copy_(interpolation_parameter * local_params.data + (1.0 - interpolation_parameter) * target_params.data)



In [112]:
# Initialise the Agent
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = Agent(state_size, action_size)

In [117]:
# Train the Agent

max_epiodes = 5000
max_steps_per_episode = 2000
score_per_batch = deque(maxlen=100)

epsilon_starting = 1.0
epsilon_ending = 0.01
epsilon_decay = 0.995
epsilon = epsilon_starting



for episode in range(1, max_epiodes+1):
  state, _ = env.reset()
  score = 0

  for steps in range(max_steps_per_episode):

    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward

    if done:
      break

  epsilon = max(epsilon_ending, epsilon_decay*epsilon)
  score_per_batch.append(score)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(score_per_batch)), end="")
  if episode % 100 == 0:
      print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(score_per_batch)))
  if np.mean(score_per_batch) >= 475.0:
      print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(score_per_batch)))
      torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
      break



Episode 100	Average Score: 34.61
Episode 200	Average Score: 70.60
Episode 300	Average Score: 109.17
Episode 400	Average Score: 103.53
Episode 500	Average Score: 174.87
Episode 600	Average Score: 287.78
Episode 700	Average Score: 241.73
Episode 800	Average Score: 220.21
Episode 900	Average Score: 221.37
Episode 1000	Average Score: 461.75
Episode 1100	Average Score: 275.28
Episode 1200	Average Score: 308.63
Episode 1300	Average Score: 322.00
Episode 1362	Average Score: 492.32
Environment solved in 1262 episodes!	Average Score: 492.32


In [118]:

# Generate the Video
import glob
import io
import base64
import imageio
from IPython.display import HTML, display

def show_video_of_model(agent, env_name):
    env = create_gym_env(env_name,  render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state, epsilon)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'CartPole-v1')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()